In [ ]:
import tempfile
from pathlib import Path
import numpy as np
from PIL import Image
from quantem.widget import Show3D
try:
    import h5py
    has_h5py = True
except Exception:
    has_h5py = False


In [ ]:
tmp = Path(tempfile.mkdtemp(prefix='show3d_loader_smoke_'))

png_dir = tmp / 'png_stack'
png_dir.mkdir()
for i in range(3):
    arr = (np.ones((32, 32), dtype=np.uint8) * (10 + i)).astype(np.uint8)
    Image.fromarray(arr).save(png_dir / f'slice_{i:02d}.png')

tiff_file = tmp / 'stack.tiff'
frames = [
    Image.fromarray((np.ones((32, 32), dtype=np.uint8) * 20).astype(np.uint8)),
    Image.fromarray((np.ones((32, 32), dtype=np.uint8) * 21).astype(np.uint8)),
]
frames[0].save(tiff_file, save_all=True, append_images=frames[1:])

mixed_dir = tmp / 'mixed_stack'
mixed_dir.mkdir()
Image.fromarray((np.ones((32, 32), dtype=np.uint8) * 30)).save(mixed_dir / 'a.png')
frames[0].save(mixed_dir / 'b.tiff', save_all=True, append_images=frames[1:])

emd_file = tmp / 'stack.emd'
if has_h5py:
    with h5py.File(emd_file, 'w') as h5f:
        h5f.create_dataset('preview/thumb', data=np.ones((16, 16), dtype=np.float32) * 2.0)
        h5f.create_dataset('data/signal', data=np.ones((2, 32, 32), dtype=np.float32) * 55.0)


In [ ]:
Show3D.from_folder(png_dir, file_type='png')
Show3D.from_tiff(tiff_file)
Show3D.from_folder(mixed_dir, file_type='png')
Show3D.from_folder(mixed_dir, file_type='tiff')
if has_h5py:
    Show3D.from_emd(emd_file, dataset_path='/data/signal')


In [ ]:
explicit_error = ''
try:
    Show3D.from_path(mixed_dir)
except Exception as exc:
    explicit_error = str(exc)
print('EXPLICIT_ERROR:', explicit_error)
